# Gene Expressions for different types of tumor
<div style="text-align: center;">
    <img src="images/gene_protein_cancer.jpg" alt="image" width="300" height="200">
</div>
[image src: https://www.cancer.gov/about-cancer/causes-prevention/genetics]

This project aims to identify different gene expressions associated to 5 types of tumor : 
- BRCA (Breast Cancer): Family of Genes (BRCA1 and BRCA2) are known as tumor suppresors. But mutation in these genes cause cancer.
- KIRC (Kidney Renal Clear Cell Carcinoma): 
- COAD (Colon Adenocarcinoma)
- LUAD (Lung Adenocarcinoma)
- PRAD (Prostate Adenocarcinoma)

The dataset is sourced from https://archive.ics.uci.edu/dataset/401/gene+expression+cancer+rna+seq.
The original dataset is published at https://www.synapse.org/Synapse:syn300013/discussion/threadId=5455. The Gene names in the dataset are dummy names. The actual gene names are at https://www.ncbi.nlm.nih.gov/gene, per this discussion thread https://www.synapse.org/Synapse:syn300013/discussion/threadId=5455. 

In [ ]:
import warnings

warnings.filterwarnings("ignore")

DATA_ANALYSIS_DIR = "data-analysis/"
MODELS_DIR = "models/"
DATA_DIR = "data/"

## Data Loading


In [ ]:
from time import time
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
from numpy import ndarray
from pandas import DataFrame
from pandas import Series
from pandas.core.indexes.base import Index


from sklearn.model_selection import train_test_split

from src.utils.ArrayUtils import get_list_of_items_in_both_lists



In [ ]:
# read the data from files
url_reconstructed = 'TCGA-PANCAN-HiSeq-801x20531/data.csv'

# url = 'https://drive.google.com/file/d/1VXyhDXpYT8G2Buhkc6kBjw93CLG1y1f0/view?usp=drive_link'
# # Use only the Id and reconstruct the URL
# url_reconstructed = 'https://drive.google.com/uc?id=' + url.split('/')[-2]

df = pd.read_csv(url_reconstructed)
tumor_df = pd.read_csv('TCGA-PANCAN-HiSeq-801x20531/labels.csv')
df

In [ ]:
df.info()


In [ ]:
print(f"Dataframe before adding the class column: {df.info()}")
print(f"Total # of columns: {len(df.columns)}")
print("COlumns type", type(df.columns))

In [ ]:
# Copy over the Class column into the main dataframe
df = df.rename(columns={'Unnamed: 0': 'sample'})
tumor_df = tumor_df.rename(columns={'Unnamed: 0': 'sample'})
df['Class'] = np.where( (df['sample'] == tumor_df['sample']), tumor_df['Class'], df['sample'])

# df['Class'] = tumor_df['Class']
# print(f"Total # of columns: {len(df.columns)}")

df.drop(columns=['sample'], axis=1, inplace=True)

df.head()


## Exploratory Data Analysis (EDA)
### Data Cleaning and PreProcessing

Data : 
- No NANs as stated on the data source page


In [ ]:
df.isnull().any().any()

In [ ]:
# Find columns with 0 values
zero_cols = df.columns[(df == 0).all()]
print(f"# of columns with all 0s in them : {zero_cols}")
df = df.drop(columns=zero_cols, axis=1)
print(f"# of columns with all 0s in them : {df.columns[(df == 0).all()]}")

In [ ]:
# Post cleanup, write the data to file
df.to_csv(DATA_ANALYSIS_DIR + "df_PostColumnCleanup.csv")


In [ ]:
# Class analysis
class_unique_vals = df['Class'].unique()
assert(len(class_unique_vals) == 5)

# print the distribution
print(f"Distribution of Class values: \n{df['Class'].value_counts()}")


In [ ]:
# Split -
X = df.drop(columns=['Class'], axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df['Class'])


#### Summary : Data Cleaning and PreProcessing : 
- 0 NaNs
- 267 columns with only 0s as values. Dropped all of these columns
- 5 target values aka classifications


### Univariate and Multivariate Analysis
Given the large # of variables (columns), charting each out against other can be a challenge. Hence printing key insights such as stats to file.

##### Statistical Description of Data

In [ ]:
from scipy import stats

df_desc = df.describe()
df_desc_t = df_desc.T
# NOTE: We use this data to generate random test data when testing from App.
df_desc_t.to_csv(DATA_ANALYSIS_DIR + "df_describe.csv")

df_desc_zscore = df_desc.apply(stats.zscore)
df_desc_zscore.T.to_csv(DATA_ANALYSIS_DIR + "rawdata_zscore.csv")
# df_desc['upperbound'] = df_desc['mean'] + 3*(df_desc['std'])
# df_desc['lowerbound'] = df_desc['mean'] - 3*(df_desc['std'])


##### Correlation
Commented out since it takes too long to execute. Not much insight could be obtained.

In [ ]:
# --- NOTE : This take quite some time to run 11+ mins
# corr_df = X.corr()
# corr_df.to_csv(DATA_ANALYSIS_DIR + 'features_corr_matrix.csv')


In [ ]:
# Are there any Features with Correlation value= NaN ?
# -- In total there are 267 cols with 0 values, since they are deleted, result is zero
# corr_nan_list = corr_df.columns[corr_df.isna().all()].tolist()
# print(corr_nan_list)
# corr_df.head()

##### Statistical Description per Class

In [ ]:
# Gene values (mean, mode, min, max, range) per class
# ---- NOTE : Below can take about 30secs to run.
# (X['gene_0'] == 0).sum()

def describeEachClass():
    for i in range(len(class_unique_vals)) :
        cls = class_unique_vals[i]
        # print('i= ', i, 'class= ', cls)
        df[df['Class'] == cls].describe().T.to_csv(f'{DATA_ANALYSIS_DIR}{cls}_describe_output.csv')

describeEachClass()
# print(type(df[df['Class'] == 'BRCA'].describe()))

##### Utility Functions

In [ ]:
# Data has columns where values are close to 100% quartile. Get a list of these columns

def get_features_with_4thquartile_values_only(df) :
    '''
    @param df : Dataframe with all the data
    @return Dataframe consisting of descriptive stats for features in  in `df` where all values fall in 4th quartile.
    '''
    #generate descriptive statistics of given dataframe
    df_desc = df.describe()
    df_desc_t = df_desc.T

    print(f"Total # of columns: {len(df_desc_t)}")
    df_cols_outliers = df_desc_t[(df_desc_t['25%'] == 0) & (df_desc_t['50%'] == 0) & (df_desc_t['75%'] == 0)]
    print(f"# of columns with only outliers: {len(df_cols_outliers)}")

    return df_cols_outliers



In [ ]:

# Identify if any of above columns have patterns such as associated with a type of tumor.
# Output the count of rows per Tumor type, for each of the above genes
def get_features_target_class_counts(df, df_cols_outliers, target_col='Class'):
    '''
    @param df : Dataframe with all the data
    @param df_cols_outliers : Dataframe with descriptive stats of features. See #get_features_with_4thquartile_values_only(df)
    @param target_col : Dependent variable in dataframe `df`.
    '''
    df_col_out_values = pd.DataFrame()
    for i in range(len(df_cols_outliers)):
        # for each record in outliers df, get only the rows that hv value greater than 0
        df_outliers_temp = df[df[df_cols_outliers.index[i]] > 0][[target_col, df_cols_outliers.index[i]]]
        df_outliers_temp1 = df_outliers_temp[target_col].value_counts().to_frame().T
        df_outliers_temp1['gene'] = df_cols_outliers.index[i]

        df_col_out_values = pd.concat([df_col_out_values, df_outliers_temp1], ignore_index=True)

        # df_col_out_values = pd.concat([df_col_out_values, df_outliers_temp], ignore_index=True)

    return df_col_out_values



##### Features with outliers

In [ ]:
df_feature_outliers = get_features_with_4thquartile_values_only(df)
df_feature_outliers_count = get_features_target_class_counts(df, df_feature_outliers, 'Class')
df_feature_outliers_count.to_csv(f'{DATA_ANALYSIS_DIR}Outliers_cols_values_count.csv')


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

# init vars to help draw the layout
max_rows = 8
max_cols = 4
layout_diviser = min(max_rows, max_cols)

# vars to help place graph in the matrix
print(len(df_feature_outliers_count))
# no of features to plot per graph determined based on graph matrix size
features_per_graph = math.ceil( len(df_feature_outliers_count) / (max_rows * max_cols) )
start_index: int = 0
end_index: int = features_per_graph

# Total # of graphs to plot
total_graphs = math.ceil(len(df_feature_outliers_count) / features_per_graph)
print(f"# of feature groups: {total_graphs}")

# init plot
fig = make_subplots(rows=max_rows, cols=max_cols, shared_xaxes=True, vertical_spacing=0.03)

col_headers = df_feature_outliers_count.columns.to_list()

# Loop through # of graphs to plot. For each graph, plot the determined # of features
for fg in range(total_graphs):
    row = (fg) // layout_diviser
    col = (fg) % layout_diviser
    # print(f"row= {row} col= {col}")
    plot_df = df_feature_outliers_count.iloc[start_index:end_index, :]
    for i in range(len(plot_df)):
        fig.add_trace(go.Scatter(x=col_headers,
                                 y=plot_df.iloc[i, :-1],
                                 mode='lines',
                                 name=plot_df.loc[plot_df.index[i], 'gene']),
                                 row=row+1, col=col+1) #add_trace row and col, start index is 1, while row and col start with 0

    start_index = start_index + features_per_graph
    end_index = end_index + features_per_graph


fig.update_layout(height=1200, width=800, title_text="Outlier columns only : Records with (non-zero) values across the 5 classes")
fig.show()


#### Summary : Univariate and Multivariate Analysis

- Descriptive Statistics of data : 1660 columns have mostly 0s, as values and their values fall into the 4th quartile.
- Outliers : Given the above 1660 columns / features have largely outliders, from above graph we identify the count of actual rows having these outlier values is less than 150, mostly less than 100. 
- Correlation : Given the large number of features, finding correlation between the features is challenging. The output file does not even open. Thus no inference to make.


### Feature Selection


### Dimensionality Reduction

#### PCA
To meet PCA needs, we need to ensure data confirms to below : 
- <u>correlation</u> between Independent vars.
- Variables must be <u>continuous</u> i.e numeric
- Variables must <u>be on same scale</u>. If one is in 1000s and another single digit, we will need to scale it.
- Data must be free from outliers

##### Utility functions

In [ ]:
# get quartiles for each column
def get_quartiles(col):
    quartile1, quartile3 = col.quantile([0.25, 0.75])
    inter_quartile_range = quartile3 - quartile1
    lower_range = quartile1 - (1.5* inter_quartile_range)
    upper_range = quartile3 + (1.5* inter_quartile_range)

    return lower_range, upper_range

# lowerQ, upperQ = get_quartiles(X_train['gene_1'])
# print(f'lower quantile= {lowerQ}, upper quantile= {upperQ}')

In [ ]:
# cols = X_train_pca.columns

def normalize_within_quartiles(df_X:DataFrame, cols:Index):
    '''
    @input df_X : Dataframe with ONLY independent variables.
    '''
    for i in cols :
        LL, UL = get_quartiles(df_X[i])
        # if (i == 'gene_0'):
        #     print(f"UL: {np.where(df_X[i] > UL, UL, df_X[i])}")
            # print(f"{np.where(df_X[i] < LL, LL, df_X[i]}")
        df_X[i] = np.where(df_X[i] > UL, UL, df_X[i])
        df_X[i] = np.where(df_X[i] < LL, LL, df_X[i])
    return df_X



In [ ]:
# print(type(df_feature_outliers.index.to_list()))
def clean_input_data(X_dataframe: DataFrame, df_feature_outliers: DataFrame):
    # print(len(X_train.columns))
    X_train_pca = X_dataframe.drop(columns=df_feature_outliers.index.to_list())
    # X_train_pca.columns
    X_train_pca = normalize_within_quartiles(X_train_pca, X_train_pca.columns)
    X_train_pca.head(10)
    print(f"Total # of cols after removing quartiles: {len(X_train_pca.columns)}")

    return X_train_pca

##### Prep for PCA

In [ ]:
# PCA -
# -- Is PCA the right approach to this data set where we are trying to identify Gene expressions for tumors.
# -- Shudnt we include every expression - small or big ?
from sklearn.decomposition import PCA

pca = PCA(random_state=42)


##### Remove Outliers
Based on data analysis on [outliers](#features-with-outliers) we know there are 1660 features with values in the 4th quartile. These gene expressions (features) are not specific to a tumor type (Class aka dependent variable) as observed in the output of function #get_features_target_class_counts. We will assume that these Gene expressions have not been recorded accurately or these have minimal influence from a tumor. Hence we remove these before running PCA.

In [ ]:
X_train_pca = clean_input_data(X_train, df_feature_outliers)
X_train_pca.head(10)

#### Summary : Feature Selection
- Features with outliers values (in 4th quartile) dropped
- PCA reduces features to 640, from 18604 features. Data prepared for PCA execution in next section, as part of the Pipeline.


## Modeling

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
import joblib



##### Utility Functions for Graphing and Analysis

In [ ]:
# Function to plot Confusion Matrix
def plot_confusion_matrix(conf_matrix:ndarray, labels:ndarray, title="Confusion Matrix"):

    conf_matrix = conf_matrix[::-1]
    # labels = labels[::-1]

    #heat map
    fig = go.Figure(data=go.Heatmap(
        z=conf_matrix,
        x=labels,
        y=labels[::-1], #reverse the order to align labels with way Conf matrix is output
        colorscale='Rainbow', # 'Hot', # 'YlOrRd', # 'YlGnBu', #'Viridis',
        texttemplate="%{z}",
        textfont={"size": 10}
    ))

    fig.update_layout(
        title_text = title,
        xaxis_title="Predicted Class",
        yaxis_title="Actual Class",
        # xaxis={'side': 'top'},
        # yaxis={'autorange': 'reversed'},
        width=500,
        height=500,
    )

    fig.show()



In [ ]:

# Using plotly's graph_objects
def plot_feature_importance_comparison_plotly(pca_model: PCA, classifier_model, feature_importances: Series, X_pre_pca_df: DataFrame):

    # top_features = feature_importances.head(10)

    # fig = px.bar(features_df, x=features_df.index, y=features_df[0])
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=feature_importances.index,
        y=feature_importances.values,
        marker=dict(color='indianred'),
        marker_color='indianred'
    ))
    fig.update_layout(title=f'Gene expression contribution to the model\n{classifier_model}', template='plotly_white')
    fig.show()



In [ ]:
def get_pca_features_weights(pca_model: PCA, classifier_model, feature_names: Index) :
    '''
    Identify the overall weights of each feature in terms of its contribution to the given model. From the given Classifier model
    it retrieves feature importances and this data is merged with the PCA components matrix.
    @param pca_model : PCA object, after it has been fit / trained on the data
    @param classifier_model : Model used for classification
    @param feature_names : List of all features used, before PCA was run.
    @return Series consisting of contribution of each feature to the model
    '''

    # get feature contributions for each Principal Component
    # feature_names = X_pre_pca_df.columns
    n_components = len(pca_model.components_)
    pca_components_df = pd.DataFrame(pca_model.components_.T,
                                     columns=[f'PC{i+1}' for i in range(n_components)],
                                     index=feature_names)


    # Random forest importances for each component
    if (hasattr(classifier_model, 'feature_importances_')):
        rf_importances = pd.Series(
            classifier_model.feature_importances_,
            index=[f'PC{i+1}' for i in range(n_components)]
        )

        # --- calculate original feature importance by weighted combination ---
        orig_importances = pca_components_df.dot(rf_importances).abs() # we only care abt the magnitude
        # sum of all shud be 1, hence find each value's contrib to 100%
        orig_importances = orig_importances / orig_importances.sum()
        return orig_importances.sort_values(ascending=False)

    return None



In [ ]:
def analyze_tree_path(pca_model: PCA, classifier_model, X_pca_df: DataFrame, y_series:Series, feature_names: Index, class_names=class_unique_vals):
    '''
    For each class / target, analyze the contribution of each feature by traversing the tree, finding the leaf node and
    then merging that with each feature's individual weights, calculated using the function #get_pca_features_weights.
    @param pca_model : PCA model, after the data has been fit aka trained on the data.
    @param classifier_model : Model used for classification, after it has been trained on the data
    @param X_pca_df : The DataFrame after PCA has been run and data has been transformed.
    @param y_series : The Y series corresponding to above X_pca_df dataset i.e if above is `test` dataset, this should also be test dataset
    @param feature_names : All the features used prior to running PCA
    @param class_names : List of unique Classes / Target Variables, the data represents. In this case we have the 5 tumors.
    @return DataFrame with columns as the 4 target classes and rows as features aka gene expressions.
    '''
    # feature_names = X_pca_df.columns
    #init dictionary that will hold each class details
    class_importances = {class_name: np.zeros(len(feature_names)) for class_name in class_unique_vals}

    #loop through each DecisionTree used by the model
    for tree in classifier_model.estimators_:
        tree_importances = tree.feature_importances_
        # get index of leaf node where sample is predicted
        leaf_nodes = tree.apply(X_pca_df)

        for row_idx, leaf_id in enumerate(leaf_nodes):
            # leaf_nodes has only the node number. Not the index that matches against corresponding index in y_series. Hence we use X_pca_df to get the index
            #row_idx is sequential increment of leaf_nodes
            record_index = X_pca_df.index[row_idx]
            predicted_class = y_series[record_index]

            original_importances = get_pca_features_weights(pca_model, classifier_model, feature_names)
            class_importances[predicted_class] += original_importances


    #Normalize
    for class_name in class_names:
        total = np.sum(class_importances[class_name])
        if total > 0:
            class_importances[class_name] /= total

    return pd.DataFrame(class_importances)



In [ ]:
# Using SHAP to explain - #TODO
import shap

def plot_SHAP(pca_model:PCA, classifier_model, X_df: DataFrame):
    explainer = shap.Explainer(classifier_model)
    shap_values = explainer(X_df) #X_pca_dataframe
    original_shap = shap_values.values @ pca_model.components_


In [ ]:
def convert_to_binary(y_series: Series):
    label_binzer = LabelBinarizer()
    label_binzer.fit(y_series)
    y_series_bin = np.array(label_binzer.transform(y_series))
    return y_series_bin


In [ ]:
def create_pipeline(model):

    pca_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', pca),
        ('model', model)
    ])
    return pca_pipeline


def create_grid_model(model, model_params: dict):
    # call create Pipeline
    pipeline = create_pipeline(model)
    # cross validation param is default = 5. n_jobs configured as param
    grid_search = GridSearchCV(estimator=pipeline, param_grid=model_params, scoring='accuracy', n_jobs=1, refit=True, verbose=1)

    return grid_search


In [ ]:
models_config = {
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'model__n_estimators':[50, 100, 200],
            'model__max_depth': [None, 10, 20],
            'model__min_samples_split': [2, 5]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
        'params': {
            'model__n_estimators':[50, 100, 200],
            'model__max_depth': [None, 10, 20],
            'model__eta': [0.2, None, 0.4],
            #'model__n_jobs': [1]
        }
    }
}

#### Random Forest

In [ ]:
# best_results = {}
def fit_model(model, model_params:dict, X_df: DataFrame, y_series:Series):
    '''
    @param model : Classifier model
    @param model_params : Params used for tuning the hyperparameters of the given model
    @param X_df : DataFrame of the independent vars with data
    @param y_series : A Series object with target class data.
    @return dict : Consisting of keys : Execution Time, Accuracy, Precision, Recall, F1-Score, Confusion matrix, Best Estimate, Best Params, Best Accuracy
    '''
    # for model_name, model_params in models_config.items():
    start_time = time()

    grid_search = create_grid_model(model=model, model_params=model_params)
    grid_search.fit(X_df, y_series)

    end_time = time()

    return grid_search



def predict_model(grid_search: GridSearchCV, X_df: DataFrame, y_series: Series):
    start_time = time()

    y_preds = grid_search.predict(X_df)

    end_time = time()

    best_results = {
        'Execution Time': (end_time - start_time),
        'Accuracy': accuracy_score(y_series, y_preds),
        'Precision': precision_score(y_series, y_preds, average='weighted'),
        'Recall': recall_score(y_series, y_preds,  average='weighted'),
        'F1-Score': f1_score(y_series, y_preds, average='weighted'),
        'Confusion matrix': confusion_matrix(y_series, y_preds),
        # 'GridSearchCV': grid_search,
        'Best Estimate': grid_search.best_estimator_,
        'Best Params': grid_search.best_params_,
        'Best Accuracy': grid_search.best_score_
    }

    return best_results



In [ ]:

# best_results = fit_predict_model(X_train_pca, y_train)

# Random Forest :
rf_model = models_config['Random Forest']['model']
rf_model_config = models_config['Random Forest']['params']
rf_grid_search = fit_model(model=rf_model, model_params=rf_model_config, X_df=X_train_pca, y_series=y_train)



##### Model Evaluation

In [ ]:
rf_X_test_pca = clean_input_data(X_test, df_feature_outliers)

rf_best_results = predict_model(grid_search=rf_grid_search, X_df=rf_X_test_pca, y_series=y_test)
print(rf_best_results)


In [ ]:
rf_conf_matrix = rf_best_results['Confusion matrix']
plot_confusion_matrix(rf_conf_matrix, labels=class_unique_vals, title='Random Forest with PCA : Confusion Matrix')


In [ ]:
# Estimator :
rf_pca_model = rf_grid_search.best_estimator_['pca']
rf_trained_model = rf_grid_search.best_estimator_['model']


In [ ]:
# Find contribution of each feature to the model
rf_pca_weights_df = get_pca_features_weights(pca_model=rf_pca_model,
                                            classifier_model=rf_trained_model,
                                            feature_names=rf_X_test_pca.columns)
if rf_pca_weights_df is not None:
    # plot_feature_importance_comparison(pca_model, rd_model, pca_weights_df, X_pca_dataframe)
    plot_feature_importance_comparison_plotly(rf_pca_model, rf_model, rf_pca_weights_df, rf_X_test_pca)
else :
    print(f'PCA Weights for {rf_trained_model} could not be calculated')


In [ ]:

rf_pca_weights_df[rf_pca_weights_df > 0][1:11]

##### Persisting the model

In [ ]:

joblib.dump(rf_trained_model, f"{MODELS_DIR}RForest_model.pkl")

In [ ]:
# test to make sure persisted model can be reloaded correctly.
rf_model_joblib: RandomForestClassifier = joblib.load(f"{MODELS_DIR}RForest_model.pkl")
print(rf_model_joblib.feature_importances_)

In [ ]:
#SHAP :
# plot_SHAP(pca_model=rf_pca_model, classifier_model=rf_trained_model, X_df=rf_X_test_pca)


#### XGBoost Model

In [ ]:
# XGB requires target vars in binary format and hence cannot reuse the fn used for RandomForest
def fit_xgb_classifier(model: XGBClassifier, model_params:dict, X_df: DataFrame, y_series:ndarray):
    '''
    @param model : Classifier model
    @param model_params : Params used for tuning the hyperparameters of the given model
    @param X_df : DataFrame of the independent vars with data
    @param y_series : An ndarray object with binary representation of target classes
    @return dict : Consisting of keys : Execution Time, Accuracy, Precision, Recall, F1-Score, Confusion matrix, Best Estimate, Best Params, Best Accuracy
    '''
    # for model_name, model_params in models_config.items():
    start_time = time()

    grid_search = create_grid_model(model=model, model_params=model_params)
    grid_search.fit(X_df, y_series)

    end_time = time()

    return grid_search


def predict_xgb_classifier(grid_search: GridSearchCV, X_df: DataFrame, y_series_bin: ndarray):
    start_time = time()

    y_preds = grid_search.predict(X_df)

    end_time = time()

    y_test_bin_arr = np.array(y_series_bin)
    # confusion matrix
    cm = confusion_matrix(y_true=np.argmax(y_test_bin_arr, axis=1), y_pred=np.argmax(y_preds, axis=1)) #, labels=class_unique_vals)
    # plot_confusion_matrix(cm, labels=class_unique_vals, title='XGBoost with PCA : Confusion Matrix')

    best_results = {
        'Execution Time': (end_time - start_time),
        'Accuracy': accuracy_score(y_series_bin, y_preds),
        'Precision': precision_score(y_series_bin, y_preds, average='weighted'),
        'Recall': recall_score(y_series_bin, y_preds,  average='weighted'),
        'F1-Score': f1_score(y_series_bin, y_preds, average='weighted'),
        'Confusion matrix': cm,
        # 'GridSearchCV': grid_search,
        'Best Estimate': grid_search.best_estimator_,
        'Best Params': grid_search.best_params_,
        'Best Accuracy': grid_search.best_score_
    }

    return best_results



In [ ]:
xgb_model = models_config['XGBoost']['model']
xgb_model_config = models_config['XGBoost']['params']

y_train_bin = convert_to_binary(y_train)
# print(y_train_bin)
# Uncomment below to identify mappings between binary format and the actual label
# 0=BRCA 1=COAD, 2=KIRC, 3=LUAD, 4=PRAD
# print("Binary data: ", y_train_bin)
# print("Label to int mapping: ", label_binzer.inverse_transform(np.array(y_train_bin)))

xgb_grid_search = fit_xgb_classifier(model=xgb_model, model_params=xgb_model_config, X_df=X_train_pca, y_series=y_train_bin)

# xgb_train_best_results = predict_xgb_classifier(grid_search=xgb_grid_search, X_df=X_train_pca, y_series_bin=y_train_bin)
# print(xgb_train_best_results)


##### Model Evaluation (XGB)

In [ ]:
xgb_y_test_bin = convert_to_binary(y_test)
xgb_X_test_pca = clean_input_data(X_test, df_feature_outliers)

xgb_best_results = predict_xgb_classifier(grid_search=xgb_grid_search, X_df=xgb_X_test_pca, y_series_bin=xgb_y_test_bin)
print(xgb_best_results)


In [ ]:
xgb_conf_matrix = xgb_best_results['Confusion matrix']
plot_confusion_matrix(xgb_conf_matrix, labels=class_unique_vals, title='XGBoost with PCA : Confusion Matrix')


In [ ]:
xgb_pca_model = xgb_grid_search.best_estimator_['pca']
xgb_trained_model = xgb_grid_search.best_estimator_['model']


In [ ]:
# Find contribution of each feature to the model
xgb_pca_weights_df = get_pca_features_weights(pca_model=xgb_pca_model,
                                            classifier_model=xgb_trained_model,
                                            feature_names=xgb_X_test_pca.columns)
if xgb_pca_weights_df is not None:
    # plot_feature_importance_comparison(pca_model, rd_model, pca_weights_df, X_pca_dataframe)
    plot_feature_importance_comparison_plotly(xgb_pca_model, xgb_trained_model, xgb_pca_weights_df, xgb_X_test_pca)
else :
    print(f'PCA Weights for {rf_trained_model} could not be calculated')


In [ ]:
# Persisting data for tests
# print(xgb_X_test_pca.head())
xgb_X_test_pca.to_csv(DATA_DIR + 'xgb_X_after_pca_dataset.csv')
print('516: ', y_test[516])
print('329: ', y_test[329])
print('52: ', y_test[52])
print('141: ', y_test[141])



# Generate statistical description of these columns to help generate random values on the App side - deprecated
len(xgb_X_test_pca.columns)
xgb_test_descr_df = xgb_X_test_pca.describe().T
# xgb_test_descr_df.to_csv(DATA_DIR + 'xgb_test_X_describe.csv')


In [ ]:
xgb_pca_model.get_feature_names_out()

##### Persisting the Model

In [ ]:
# Persist just the XGBooster model. If used the inputs would be 640 PCA components which can be identified using xgb_pca_model.get_feature_names_out()
joblib.dump(xgb_trained_model, f"{MODELS_DIR}XGBoost_model.pkl")

# persis the whole trained GridSearchCV
joblib.dump(xgb_grid_search, f"{MODELS_DIR}xgb_GridSearch_Pipeline.pkl")

In [ ]:
# test to make sure persisted model can be reloaded correctly.
# xgb_model_joblib: XGBClassifier = joblib.load(f"{MODELS_DIR}XGBoost_model.pkl")
# print(xgb_model_joblib.feature_importances_)

xgb_gridcv_pipeline_joblib: GridSearchCV = joblib.load(f"{MODELS_DIR}xgb_GridSearch_Pipeline.pkl")
xgb_gridcv_pipeline_joblib.best_estimator_

In [ ]:
# Unit testing : to make sure model prediction is same as when input from Web App
unittest_data_X = pd.read_csv(DATA_ANALYSIS_DIR + 'request.csv')
# print("Printing req data: \n", type(unittest_data_X.iloc[0:1, 1:]))
unittest_data_pred = xgb_grid_search.predict(unittest_data_X.iloc[0:1, 1:])

# unittest_data_pred = xgb_grid_search.predict(xgb_X_test_pca.iloc[0:1, 0:])
unittest_data_pred

### Comparing the 2 Classifier Models

In [ ]:
results_df = pd.DataFrame([rf_best_results, xgb_best_results]).T #.sort_values(by='Accuracy', ascending=False)
results_df.columns = ['Random Forest', 'XGBoost']
results_df

### Code Under Construction
Please ignore below section as this code is being worked on

In [ ]:
# Area under Curve - REF : https://www.geeksforgeeks.org/interpreting-random-forest-classification-results/
# -------- #TODO : need to fix the roc_curve function
def plot_aoc_randomforest(rd_test_pred_proba: ndarray, y_test: Series) :
    target_vals = y_test.unique()
    # y_test_bin = label_binarize(y_test, classes=[0,1,2,3,4])
    label_binzer = LabelBinarizer()
    label_binzer.fit(y_test)
    y_test_bin = np.array(label_binzer.transform(y_test))
    print(y_test_bin[0])
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    print(f"y_test_bin : {y_test_bin[1]}")
    print(f"rd_preds_prob : {rd_test_pred_proba[:, 1]}")
    # print(f"y_test_bin : {len(y_test_bin[:, 1])}")
    # print(f"rd_preds_prob : {rd_preds_prob[:, 1]}")
    # print(rd_preds_prob)

    for index in range(len(target_vals)):
        fpr[index], tpr[index], _ = roc_curve(y_test_bin[index], rd_test_pred_proba[:, index])
        # print(f"FPR at {index}: \n{fpr[index]}")
        # print(f"TPR at {index}: \n{tpr[index]}")
        roc_auc[index] = auc(fpr[index], tpr[index])

    # Plot ROC curve
    # plt.figure()
    # for index in range(len(target_vals)) :
    #     plt.plot(fpr[index], tpr[index], lw=2, label=f"ROC curve of class {target_vals[index]} (area = {roc_auc[index]:.2f})")

    # # plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
    # plt.xlim([0.0, 1.0])
    # plt.ylim([0.0, 1.05])
    # plt.xlabel('False Positive Rate')
    # plt.ylabel('True Positive Rate')
    # plt.title('Receiver Operating Characterstic for Tumor classes')
    # plt.legend(loc="lower right")
    # plt.show()

